# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from Kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications, we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 

Used car prices can be driven based on many factors such as car brands, year of manufactue, number of miles driven etc

Other factors such as what month of the year the car is being sold can also have significant bearing on demand vs supply which inturn drives cost of the car

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [2]:
#Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.tsa.arima.model import ARIMA

In [5]:
# Load dataset
df = pd.read_csv('data/vehicles.csv')
df.head()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
0,7222695916,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az
1,7218891961,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar
2,7221797935,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl
3,7222270760,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma
4,7210384030,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc


In [6]:
#Do EDA and some basic data quality checks

# 1. Basic info
print("===== Basic Info =====")
print(df.info())

# 2. Summary statistics for numeric columns
print("\n===== Summary Statistics =====")
print(df.describe())

# 3. Missing values analysis
print("\n===== Missing Values =====")
missing_values = df.isnull().sum().sort_values(ascending=False)
print(missing_values)

# 4. Percentage of missing values
missing_percentage = (missing_values / len(df)) * 100
print("\n===== Missing Values Percentage =====")
print(missing_percentage)

# 5. Duplicate rows
duplicates = df.duplicated().sum()
print(f"\n===== Duplicate Rows =====\nTotal duplicates: {duplicates}")

# 6. Outlier detection for numeric columns (price, year, odometer)
print("\n===== Outlier Detection =====")
for col in ['price', 'year', 'odometer']:
    if col in df.columns:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        outliers = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
        print(f"Column: {col}, Outliers: {outliers}, Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")

# 7. Unique values per column
print("\n===== Unique Values per Column =====")
unique_counts = df.nunique().sort_values(ascending=False)
print(unique_counts)

# 8. Check for zero or negative values in price and odometer
print("\n===== Zero or Negative Values =====")
for col in ['price', 'odometer']:
    if col in df.columns:
        zero_negative = (df[col] <= 0).sum()
        print(f"Column: {col}, Zero/Negative count: {zero_negative}")


===== Basic Info =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-

### Data Preparation

After our initial exploration and fine-tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

Approach :

We will be using Regression model which can only take numeric attributes. And the attributes of interest are year and odometer reading and target variable is Price

In [8]:
# Clean data for regression
df = df.dropna(subset=['price','year','odometer'])
df = df[(df['price']>100)&(df['price']<100000)]
X = df[['year','odometer']]
y = df['price']

### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

<span style="font-size:20px"> Regression Model with Hyperparameter Tuning </span>

In [16]:
# Split data
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# Define model and parameter grid
model = LinearRegression()
param_grid = {
    'fit_intercept':[True,False]
}

grid = GridSearchCV(model,param_grid,cv=5)
grid.fit(X_train,y_train)

# Best parameters
print('Best Parameters:',grid.best_params_)

# Predict using best model
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)
print('RMSE:',mean_squared_error(y_test,y_pred))
print('R2 Score:',r2_score(y_test,y_pred))

Best Parameters: {'fit_intercept': True}
RMSE: 180139456.03754273
R2 Score: 0.13477641496103065


### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high-quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight into drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

In [17]:

readme_content = """# What Drives the Price of a Car?

## Summary of Findings
This project analyzes a dataset of 426K vehicles to identify factors influencing car prices. Key drivers include:
- **Year**: Newer cars generally cost more.
- **Odometer**: Lower mileage increases value.
- **Manufacturer**: Luxury brands command higher prices.
- **Condition and fuel type** also impact price.



_IncompleteInputError: incomplete input (3717599789.py, line 1)

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine-tuning their inventory.